# 資料訓練模組 - FANtaci

## 前置套件安裝
在Python3.6, pip3.6 的環境下安裝所需套件

In [ ]:
!pip3.6 install requests
!pip3.6 install pandas
!pip3.6 install numpy
!pip3.6 install scipy
!pip3.6 install sklearn

## 程式碼
主控程式如下 引用了自寫的 爬蟲模組 `FanPageCrawler` 以及 `ModelGenerator`

In [5]:
from crawler import FanPageCrawler
from modeltrain import ModelGenerator
import json

class FanpageAnalyzer:
    """
    粉絲團貼文分析工具
    """
    def __init__(self, facebook_access_token, fan_page_id, output_file_path):
        self.output_file_path = output_file_path
        self.post_limit = None
        self.comment_limit = None
        self.crawler = FanPageCrawler(facebook_access_token, fan_page_id)
        self.model_generator = ModelGenerator(output_file_path)

    def fetch_training_data_set(self, since, until, post_limit=-1, comment_limit=-1):
        """
        抓取訓練資料
        :param since: 抓取開始日期
        :param until: 抓取結束日期
        :param post_limit: 欲抓取之貼文數量 預設值為-1(為全部抓取)
        :param comment_limit: 欲抓取之回文ID數量 預設值為-1(全部抓取)
        :return: 無
        """
        self.crawler.crawl_posts_and_save(
            output_file_path=self.output_file_path,
            posts_limit=post_limit,
            comments_limit=comment_limit,
            since=since,
            until=until)

    def train_model(self, post_content=1.0, post_comment_ids=1.0, post_writer_ids=1.0, time_interval=1.0):
        """
        訓練預測模型
        """
        self.model_generator.train(post_content, post_comment_ids, post_writer_ids, time_interval)

    def analyze_visitor_posts(self, since, until):
        """
        分析訪客文章是否值得回應
        :param since: 分析開始日期
        :param until: 分析結束日期
        :return: 無
        """
        posts = self.crawler.crawl_posts(since=since, until=until)
        posts = json.loads(json.dumps(posts, default=lambda o: o.__dict__, sort_keys=True, indent=4, 
                          ensure_ascii=False))
        data_set = self.model_generator._transform_input_matrix(data=posts, partition=1)
        data_frame=self.model_generator.predict(data_set)
        self.model_generator.store_predict_output(data_frame,"{since}to{until}prediction.csv"
                                                  .format(since=since,until=until))


## 參數設定
設定存取TOKEN：該TOKEN會過期 因此需要定期更新

In [6]:
# 訓練資料來源
output_file_path = "/Users/Steve/PycharmProjects/Phantacy/result_all.json"
# 粉絲團ID
fan_page_id = "818773044827025"
# 存取TOKEN
access_token = "EAACEdEose0cBAMRX5x5uEpGIBxW6z6CsSZBhDajGvSIt4rarKOmOeXcfDXfEVEZBcdZB9X2Pgs0ut29TBxFe4G05CWalt8DRsAYqHYerM1PG1wKkTFbIQIyXZCbZABTHjutVOWuWjV5dFNouokVB4FWonIr5eMyCqCjCZAfcKojen6dWJ1mKakDuvlmMqSdSAlEtYemVVm1AZDZD"
# 宣告主控程式物件
analyzer = FanpageAnalyzer(facebook_access_token=access_token,
                           fan_page_id=fan_page_id,
                           output_file_path=output_file_path)

當主控程式物件建立之後 可以分別進行以下三種動作

## 抓取訓練集
抓取全時間範圍的所有粉絲團訪客貼文以及回文ID
建議不要抓取太近範圍(近一個月)之訪客貼文，以免影響訓練效能

In [ ]:
# 抓取開始日期
since="2015-06-01"
# 抓取結束日期
until="2017-11-01"
analyzer.fetch_training_data_set(since, until)

## 訓練程式
 - 使用Naive Bayes作為訓練模型，預測模型執行完畢之後儲存於記憶體中
 - 訓練資料和測試資料採用80/20法則，80%的資料拿來訓練，20%的資料用於驗證準確度
 - 本訓練模型包含四種特徵，可透過設定權重，決定是否採用，0表不採用

In [7]:
# 文章內容權重
post_content_weight = 1.5
# 評論者ID權重
post_comment_ids_weight = 0
# 留言者ID權重
post_writer_ids_weight = 1.0
# 時間間隔權重
time_interval_weight = 1.0
# 開始訓練模型
analyzer.train_model(post_content=post_content_weight,
                          post_comment_ids=post_comment_ids_weight,
                          post_writer_ids=post_writer_ids_weight,
                          time_interval=time_interval_weight)

轉換資料至訓練模型之輸入格式
訓練資料總大小: 6987筆
訓練集大小: 5589
測試集大小: 1398
模型訓練中 請耐心等待
MultinomialNB(alpha=2, class_prior=None, fit_prior=True)
萃取貼文者ID特徵向量中
將貼文者ID轉換為向量中
萃取時間間隔向量中
時間間隔轉換為向量中
模型預測資訊：
將貼文者ID轉換為向量中
時間間隔轉換為向量中
留言：


,實際數據,是否該回應,留言
0,False,False,署長:要警察穿這樣……先把衛生局廢了，他們的預算給警察，薪水給警察，砍警察很會，用警察很會，...
1,False,False,桃園市政府警察局大園分局竹圍所警員李冠毅11月20日20-22時備勤勤務，於22時許在大園區...
2,False,False,不要再相信台灣的鬼司法全力支持警察大罷工本人願出錢出力
3,False,False,安慰👉上香👉公祭👉忘記👉👎👎👎👎👎👎警政署的長官不是不能3班制，是你們習慣身背血腥味才睡的安...
4,False,False,不好意思，我是個局外人，當然我也知道2017臺北世界大學運動會對台灣非常重要，可是對我來說，...
5,False,False,以下是自稱板橋派出所警員ReiyuWang在靠北警察的留言，個人認為那位警員先生壓力太大，不...
6,True,True,請問署有否函發署轄各單位辦理人事座談會?目的為何?有否参加人員限制?
7,True,False,9月6日訪客留言中有位陳語書的一篇留言提到「警界退休潮一波接著一波，究因除了個人生涯規劃外，...
8,False,False,■綺夢園親子社團■向■保三總隊警犬隊■致謝:謝謝閻隊長帶領著優秀團隊，讓孩子們在反毒宣導上有...
9,True,False,【臺南市員警豪雨中救助迷途老婦，署駐區督察前往慰勉表揚】臺南市政府警察局第五分局和緯派出所警...


準確率: 68.03%


## 預測模組
 - 將執行完畢儲存於記憶體中之預測模型來分析特定時間之貼文，並給予標籤，讓使用者決定是否回文
 - 建議預測時間間隔開始與結束不超過六個月

In [8]:
# 預測開始日期
since="2017-11-01"
# 預測結束日期
until="2017-11-15"
analyzer.analyze_visitor_posts(since, until)

開始抓取資料
https://graph.facebook.com/v2.10/818773044827025/?fields=visitor_posts.limit(50).order(chronological).since(2017-11-01).until(2017-11-15){comments.limit(50){from},created_time,message,from,permalink_url}&access_token=EAACEdEose0cBAMRX5x5uEpGIBxW6z6CsSZBhDajGvSIt4rarKOmOeXcfDXfEVEZBcdZB9X2Pgs0ut29TBxFe4G05CWalt8DRsAYqHYerM1PG1wKkTFbIQIyXZCbZABTHjutVOWuWjV5dFNouokVB4FWonIr5eMyCqCjCZAfcKojen6dWJ1mKakDuvlmMqSdSAlEtYemVVm1AZDZD
目前 50 貼文查詢請求已傳送.
總共 81 貼文查詢請求已傳送.
總共 3 則評論已從貼文[3]抓取
總共 32 則評論已從貼文[5]抓取
總共 27 則評論已從貼文[8]抓取
總共 3 則評論已從貼文[9]抓取
總共 2 則評論已從貼文[12]抓取
總共 3 則評論已從貼文[13]抓取
總共 2 則評論已從貼文[14]抓取
總共 49 則評論已從貼文[15]抓取
總共 8 則評論已從貼文[17]抓取
總共 1 則評論已從貼文[18]抓取
總共 1 則評論已從貼文[19]抓取
總共 1 則評論已從貼文[21]抓取
總共 1 則評論已從貼文[22]抓取
總共 28 則評論已從貼文[23]抓取
總共 2 則評論已從貼文[25]抓取
總共 2 則評論已從貼文[27]抓取
總共 2 則評論已從貼文[28]抓取
總共 1 則評論已從貼文[30]抓取
總共 10 則評論已從貼文[35]抓取
總共 4 則評論已從貼文[37]抓取
總共 1 則評論已從貼文[38]抓取
總共 3 則評論已從貼文[40]抓取
總共 1 則評論已從貼文[43]抓取
總共 2 則評論已從貼文[46]抓取
總共 7 則評論已從貼文[49]抓取
總共 1 則評論已從貼文[50]抓取
總共 3 則評論已從貼文[54]抓取
總共 3 則評論已從貼文[56]抓取

將貼文者ID轉換為向量中
時間間隔轉換為向量中
留言：


,是否該回應,留言
0,False,請問署長現在是戒嚴嗎?
1,False,
2,False,黃國昌及其領導的政黨，對於軍公教，尤其警察最不友善。如果全國軍公教不能一鼓作氣，在12/16...
3,False,"松山分局的呂主任一句：""自己人""！後來才說，是要講對方服務態度的問題。這............."
4,False,◎員警因公受傷，代表署長慰問案◎臺北市政府警察局松山分局三民派出所巡佐鄭宗龍（男，65年次）...
5,False,世間相遇皆有緣 能為母子因萬千 今天白天沒班，獨自上大坑步道，種秋色爭奇奪艷...
6,True,
7,False,不要只給一個官方回覆，如果是一般人這種態度 應該被上銬了吧，沒想到所長還到場了解...
8,False,網路賭博遊戲。滿貫大亨現金交易買賣那麼大為何都不抓 然而只抓一些家庭衛生麻將 真的奇怪難道真...
9,False,保六員警因公受傷，代表署長慰問案：106年11月12日10時45分，保六總隊一大一中警員侯志...
